**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Mar 31 04:57:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'12271882_Q2 (1).pdf'	    'AI_Ass1_71 (1).docx'
 12271882_Q2_AI.pdf	     AI_Ass1_71.docx
 12271882_Q2_CSL.pdf	     AI_Ass1_71.pdf
 12271882_Q2_MAD.pdf	    'Andeoid architecture.pdf'
 12271882_Q2.pdf	    'android features and dvm.pdf'
'12271882_Q3 (1).pdf'	     ASL_71_Practical_4.docx
 12271882_Q3_AI.pdf	     ASL_71_Practical_5.docx
 12271882_Q3_CSL.pdf	     ASL_71_Practical_6.docx
 12271882_Q3_MAD.pdf	     ASL_71_Practical_7.docx
 12271882_Q3.pdf	     ASL_71_Practical_8.docx
 71Activity_Exp_1.docx	    'Assignment_1_IS_71 (1).docx'
 71_AI_Exp_1.docx	     Assignment_1_IS_71.docx
'71_AI_EXP_4 (1).docx'	     Assignment_2_IS_71.docx
'71_AI_EXP_4 (2).docx'	     Assignment_3_IS_71.docx
 71_AI_EXP_4.docx	    'Assignment No 2.docx'
 71_AI_EXP_5.docx	     classes.txt
 71_CSL_ASS1.docx	     Classroom
'71_END_1 (1).docx'	     Custom
 71_END_1.docx		    'difference between.pdf'
 71_END_2.docx		     Experiment_5_71.docx
 71_END_3.docx		    'Experiment no 4.gdoc'
 71_END_EX

In [6]:
%cd /mydrive/Yolo/train_yolo_to_detect_custom_object/

/content/gdrive/My Drive/Yolo/train_yolo_to_detect_custom_object


**1) Clone the Darknet**



In [ ]:
# !git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14748, done.
remote: Total 14748 (delta 0), reused 0 (delta 0), pack-reused 14748
Receiving objects: 100% (14748/14748), 13.29 MiB | 5.63 MiB/s, done.
Resolving deltas: 100% (10031/10031), done.
Checking out files: 100% (2023/2023), done.


**2) Compile Darknet using Nvidia GPU**


In [7]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/gdrive/My Drive/Yolo/train_yolo_to_detect_custom_object/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iterati

**3) Configure Darknet network for training YOLO V3**

In [ ]:
#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
#!mkdir /mydrive/train_yolo_to_detect_custom_object/yolov3

In [ ]:
#!echo "Sunglasses" > data/obj.names
#!echo -e 'classes= 1\ntrain  = /mydrive/train_yolo_to_detect_custom_object/yolov3/data/train.txt\nvalid  = /mydrive/train_yolo_to_detect_custom_object/yolov3/data/test.txt\nnames = /mydrive/train_yolo_to_detect_custom_object/yolov3/data/obj.names\nbackup = /mydrive/train_yolo_to_detect_custom_object/yolov3/' > /mydrive/train_yolo_to_detect_custom_object/yolov3/data/obj.data
#%cd /content/gdrive/My\ Drive/Yolo/train_yolo_to_detect_custom_object/yolov3/train_test/
#!mkdir data/obj

/bin/bash: /mydrive/train_yolo_to_detect_custom_object/yolov3/data/obj.data: No such file or directory
mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
#!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-03-30 08:14:20--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  17.4MB/s    in 9.1s    

2021-03-30 08:14:30 (17.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
#!unzip /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test.zip -d /mydrive/train_yolo_to_detect_custom_object/yolov3

Archive:  /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test.zip
   creating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/1.jpg  
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/1.txt  
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/10.jpg  
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/10.txt  
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/100.jpg  
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/100.txt  
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/11.jpg  
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/11.txt  
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/12.jpg  
  inflating: /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/12.txt  
  inflat

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.383500 0.294956 0.275000 0.355263

['0', '0.383500', '0.294956', '0.275000', '0.355263']
0 /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/1.txt
0 0.383500 0.294956 0.275000 0.355263
0 0.422667 0.211000 0.747333 0.254667

['0', '0.422667', '0.211000', '0.747333', '0.254667']
1 /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/10.txt
0 0.422667 0.211000 0.747333 0.254667
0 0.512167 0.829875 0.835000 0.242250

['0', '0.512167', '0.829875', '0.835000', '0.242250']
2 /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/100.txt
0 0.512167 0.829875 0.835000 0.242250
0 0.467833 0.869000 0.715000 0.259333

['0', '0.467833', '0.869000', '0.715000', '0.259333']
3 /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/11.txt
0 0.467833 0.869000 0.715000 0.259333
0 0.523500 0.516833 0.743000 0.292333

['0', '0.523500', '0.516833', '0.743000', '0.292333']
4 /mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/12.txt
0 0.523500 0.516833 0.743000

In [ ]:
import glob
images_list = glob.glob("/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/*.jpg")
print(images_list)

['/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/1.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/10.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/100.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/11.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/12.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/13.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/14.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/15.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/16.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/17.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/18.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/19.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/2.jpg', '/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/2

In [ ]:
#Create training.txt file
file = open("/mydrive/train_yolo_to_detect_custom_object/yolov3/train_test/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
!pwd

/content/gdrive/My Drive/train_yolo_to_detect_custom_object/darknet


**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train ./data/obj.data ./cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 85371, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.800426), count: 4, class_loss = 0.679101, iou_loss = 0.111924, total_loss = 0.791025 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 85375, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.831960), count: 4, class_loss = 0.757627, iou_loss = 0.077943, total_loss = 0.835570 
v3 (mse